In [1]:
%load_ext autoreload
import os
import sys

os.environ['TRANSFORMERS_CACHE'] = '/mnt/swordfish-pool2/milad/hf-cache-new'
os.environ['HF_DATASETS_CACHE'] = '/mnt/swordfish-pool2/milad/hf-cache-new'
os.environ["OPENAI_API_KEY"]= 'xxx'
os.environ['CUDA_VISIBLE_DEVICES'] = '6'
sys.path.append('./src-py')

In [4]:
import json
keys = json.load(open('/local/nlp/milad/code/llms-as-science-communicators/keys.json'))
for key, val in keys.items():
    os.environ[key] = val

In [5]:
import datasets
import json
import os
import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt
import re
from collections import Counter

pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


%autoreload
import utils
import datadreamer_generation

import tiktoken
from transformers import AutoTokenizer

/mnt/swordfish-pool2/milad/conda-envs/datadreamer/lib/python3.12/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [6]:
from huggingface_hub import login

login(keys['hf_token'])

In [7]:
output_dir = '/mnt/swordfish-pool2/milad/communicating-science-to-the-public/'

In [8]:
gpt_tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")
llama_tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")

### Generate the final conversation dataset

In [7]:
training_ds = datasets.load_from_disk(output_dir + "/high_quality_news_dataset")

In [8]:
len(training_ds)

19812

In [9]:
all_prompts = utils.get_prompt_compositions()
used_prompt = all_prompts[1]

In [ ]:
# output_dir = '/mnt/swordfish-pool2/milad/communicating-science-to-the-public/llama3-final-conv-ds/'
# resulted_ds = datadreamer_generation.generate_conversation(output_dir, 'llama3', training_ds, used_prompt, llama_tokenizer, max_input_tokens=5000)
# resulted_ds.save_to_disk(output_dir)

In [104]:
# output_dir = '/mnt/swordfish-pool2/milad/communicating-science-to-the-public/gpt3-final-conv-ds/'
# resulted_ds = datadreamer_generation.generate_conversation(output_dir, 'gpt-3', training_ds, used_prompt, gpt_tokenizer)
# resulted_ds.save_to_disk(output_dir)

In [ ]:
output_dir = '/mnt/swordfish-pool2/milad/communicating-science-to-the-public/deepseek-final-conv-ds/'
resulted_ds = datadreamer_generation.generate_conversation(output_dir, 'deepseek-r1', training_ds, used_prompt, gpt_tokenizer, max_input_tokens=5000)
resulted_ds.save_to_disk(output_dir)

[ 🤖 DataDreamer 💤 ] Initialized. 🚀 Dreaming to folder: /mnt/swordfish-pool2/milad/communicating-science-to-the-public/deepseek-final-conv-ds/composite-na-pr-guided
[ 🤖 DataDreamer 💤 ] Step 'documents' was previously run and saved, but was outdated. 😞
[ 🤖 DataDreamer 💤 ] Step 'documents' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'documents' finished and is saved to disk. 🎉
[ 🤖 DataDreamer 💤 ] Step 'documents (map)' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'documents (map)' will run lazily. 🥱
/local/nlp/milad/code/communicating-science-to-the-public/src/datadreamer_generation.py:159: UserWarning: You did not specify `total_num_rows`, so we cannot automatically update the progress % for this step. Either specify map(..., total_num_rows=#) or, to disable this warning, specify map(.., auto_progress = False)
  datasource = datasource.map(lambda row: {'inputs_len': len(encoding.encode(row['inputs']))})
[ 🤖 DataDreamer 💤 ] Step 'documents (map) (map)' is running. ⏳
[ 🤖 DataDreamer 💤 ] Step 'documen

### Cleaning up the corpus:

In [10]:
deepseek_conv_ds = datasets.load_from_disk('/mnt/swordfish-pool2/milad/communicating-science-to-the-public/deepseek-final-conv-ds')

In [11]:
deepseek_conv_ds

Dataset({
    features: ['id', 'pr-title', 'pr-article', 'pr-summary', 'sc-title', 'sc-article', 'sc-abstract', 'sc-section_names', 'sc-sections', 'sc-authors', 'source', 'Topic', 'Citation', 'Paper_URL', 'News_URL', 'pr-summary-and-article', 'parsed-pr-article', 'societal_eval_prompt_scoring_parsed', 'scientific_eval_prompt_scoring_parsed', 'clarity_eval_prompt_scoring_parsed', 'avg_quality_score', 'sc-intro', 'inputs', 'inputs_len', 'conversation', 'parsed_conv'],
    num_rows: 19452
})

In [12]:
deepseek_conv_ds = deepseek_conv_ds.filter(lambda row: '</think>' in row['conversation'])

In [13]:
deepseek_conv_ds

Dataset({
    features: ['id', 'pr-title', 'pr-article', 'pr-summary', 'sc-title', 'sc-article', 'sc-abstract', 'sc-section_names', 'sc-sections', 'sc-authors', 'source', 'Topic', 'Citation', 'Paper_URL', 'News_URL', 'pr-summary-and-article', 'parsed-pr-article', 'societal_eval_prompt_scoring_parsed', 'scientific_eval_prompt_scoring_parsed', 'clarity_eval_prompt_scoring_parsed', 'avg_quality_score', 'sc-intro', 'inputs', 'inputs_len', 'conversation', 'parsed_conv'],
    num_rows: 14772
})

In [14]:
deepseek_conv_ds.save_to_disk('/mnt/swordfish-pool2/milad/communicating-science-to-the-public/deepseek-final-conv-ds-cleaned')

Saving the dataset (0/4 shards):   0%|          | 0/14772 [00:00<?, ? examples/s]

In [14]:
deepseek_conv_ds.push_to_hub("miladalsh/deepseek-final-conv-ds-cleaned")

Uploading the dataset shards:   0%|          | 0/4 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/miladalsh/deepseek-final-conv-ds-cleaned/commit/a3ba7e41095bd963a5f782bb05da7fa2f4068ee4', commit_message='Upload dataset', commit_description='', oid='a3ba7e41095bd963a5f782bb05da7fa2f4068ee4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/miladalsh/deepseek-final-conv-ds-cleaned', endpoint='https://huggingface.co', repo_type='dataset', repo_id='miladalsh/deepseek-final-conv-ds-cleaned'), pr_revision=None, pr_num=None)

In [15]:
deepseek_conv_ds

Dataset({
    features: ['id', 'pr-title', 'pr-article', 'pr-summary', 'sc-title', 'sc-article', 'sc-abstract', 'sc-section_names', 'sc-sections', 'sc-authors', 'source', 'Topic', 'Citation', 'Paper_URL', 'News_URL', 'pr-summary-and-article', 'parsed-pr-article', 'societal_eval_prompt_scoring_parsed', 'scientific_eval_prompt_scoring_parsed', 'clarity_eval_prompt_scoring_parsed', 'avg_quality_score', 'sc-intro', 'inputs', 'inputs_len', 'conversation', 'parsed_conv'],
    num_rows: 14772
})

In [17]:
deepseek_conv_ds['Topic'][:10]

['Medicine',
 'Medicine',
 'Medicine',
 'Nano',
 'Medicine',
 'Earth',
 'Biology',
 'Medicine',
 'Chemistry',
 'Biology']